In [4]:
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [5]:
import pandas as pd

df = pd.read_csv('reduced_covid_sexe.csv')
df = df.iloc[2:]

df.head(15)

,Unnamed: 0,hosp,hosp.1,hosp.2,rea,rea.1,rea.2,rad,rad.1,rad.2,dc,dc.1,dc.2
2,2020-03-18,2972.0,1685.0,1248.0,771.0,525.0,239.0,816.0,432.0,379.0,218.0,121.0,96.0
3,2020-03-19,4073.0,2256.0,1632.0,1002.0,689.0,284.0,1180.0,580.0,562.0,327.0,179.0,136.0
4,2020-03-20,5226.0,2912.0,2134.0,1297.0,905.0,362.0,1587.0,777.0,764.0,450.0,250.0,190.0
5,2020-03-21,5900.0,3298.0,2401.0,1453.0,1018.0,401.0,1811.0,881.0,888.0,525.0,297.0,219.0
6,2020-03-22,6954.0,3906.0,2815.0,1674.0,1176.0,461.0,2117.0,1031.0,1040.0,632.0,356.0,263.0
7,2020-03-23,8673.0,4882.0,3532.0,2080.0,1465.0,569.0,2567.0,1270.0,1247.0,860.0,497.0,350.0
8,2020-03-24,10163.0,5843.0,4149.0,2503.0,1773.0,694.0,3281.0,1609.0,1624.0,1100.0,661.0,425.0
9,2020-03-25,12072.0,7064.0,4856.0,2935.0,2127.0,785.0,4085.0,2000.0,2028.0,1388.0,818.0,557.0
10,2020-03-26,13879.0,8152.0,5570.0,3351.0,2437.0,888.0,4947.0,2424.0,2456.0,1696.0,1002.0,681.0
11,2020-03-27,15701.0,9288.0,6248.0,3758.0,2735.0,991.0,5698.0,2820.0,2806.0,1995.0,1175.0,809.0


In [6]:
# créer une colomne season et l' ajouter une colomne season au dataframe 

df = df.iloc[2:]
dates = pd.to_datetime(df['Unnamed: 0'])

#définir les quatres saisons 
def get_season(dates):
    year = str(dates.year)
    seasons = {'spring': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'summer': pd.date_range(start='21/06/'+year, end='20/09/'+year),
               'autumn': pd.date_range(start='21/09/'+year, end='20/12/'+year)}
    if dates in seasons['spring']:
        return 'spring'
    if dates in seasons['summer']:
        return 'summer'
    if dates in seasons['autumn']:
        return 'autumn'
    else:
        return 'winter'

#ajouter la colone season au dataframe
df['season'] = dates.map(get_season)
 
df

,Unnamed: 0,hosp,hosp.1,hosp.2,rea,rea.1,rea.2,rad,rad.1,rad.2,dc,dc.1,dc.2,season
4,2020-03-20,5226.0,2912.0,2134.0,1297.0,905.0,362.0,1587.0,777.0,764.0,450.0,250.0,190.0,winter
5,2020-03-21,5900.0,3298.0,2401.0,1453.0,1018.0,401.0,1811.0,881.0,888.0,525.0,297.0,219.0,spring
6,2020-03-22,6954.0,3906.0,2815.0,1674.0,1176.0,461.0,2117.0,1031.0,1040.0,632.0,356.0,263.0,spring
7,2020-03-23,8673.0,4882.0,3532.0,2080.0,1465.0,569.0,2567.0,1270.0,1247.0,860.0,497.0,350.0,spring
8,2020-03-24,10163.0,5843.0,4149.0,2503.0,1773.0,694.0,3281.0,1609.0,1624.0,1100.0,661.0,425.0,spring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,2023-03-27,13113.0,5950.0,6923.0,728.0,457.0,263.0,861972.0,421189.0,432745.0,135936.0,77555.0,57324.0,spring
1107,2023-03-28,13178.0,6000.0,6943.0,720.0,452.0,261.0,862458.0,421407.0,433007.0,135975.0,77575.0,57342.0,spring
1108,2023-03-29,13184.0,5989.0,6964.0,734.0,456.0,270.0,862962.0,421645.0,433268.0,136009.0,77594.0,57356.0,spring
1109,2023-03-30,13186.0,5953.0,7003.0,732.0,459.0,267.0,863331.0,421833.0,433447.0,136045.0,77609.0,57376.0,spring


In [7]:
#encoder la colomne unnamed: 0 datetime 
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['Unnamed: 0'] = labelencoder.fit_transform(df['Unnamed: 0'])

df

,Unnamed: 0,hosp,hosp.1,hosp.2,rea,rea.1,rea.2,rad,rad.1,rad.2,dc,dc.1,dc.2,season
4,0,5226.0,2912.0,2134.0,1297.0,905.0,362.0,1587.0,777.0,764.0,450.0,250.0,190.0,winter
5,1,5900.0,3298.0,2401.0,1453.0,1018.0,401.0,1811.0,881.0,888.0,525.0,297.0,219.0,spring
6,2,6954.0,3906.0,2815.0,1674.0,1176.0,461.0,2117.0,1031.0,1040.0,632.0,356.0,263.0,spring
7,3,8673.0,4882.0,3532.0,2080.0,1465.0,569.0,2567.0,1270.0,1247.0,860.0,497.0,350.0,spring
8,4,10163.0,5843.0,4149.0,2503.0,1773.0,694.0,3281.0,1609.0,1624.0,1100.0,661.0,425.0,spring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,1102,13113.0,5950.0,6923.0,728.0,457.0,263.0,861972.0,421189.0,432745.0,135936.0,77555.0,57324.0,spring
1107,1103,13178.0,6000.0,6943.0,720.0,452.0,261.0,862458.0,421407.0,433007.0,135975.0,77575.0,57342.0,spring
1108,1104,13184.0,5989.0,6964.0,734.0,456.0,270.0,862962.0,421645.0,433268.0,136009.0,77594.0,57356.0,spring
1109,1105,13186.0,5953.0,7003.0,732.0,459.0,267.0,863331.0,421833.0,433447.0,136045.0,77609.0,57376.0,spring


In [ ]:
#afficher les type de données du dataframe
df.dtypes

Unnamed: 0         int64
hosp             float64
hosp.1           float64
hosp.2           float64
rea              float64
rea.1            float64
rea.2            float64
rad              float64
rad.1            float64
rad.2            float64
dc               float64
dc.1             float64
dc.2             float64
season_autumn       bool
season_spring       bool
season_summer       bool
season_winter       bool
dtype: object

In [ ]:
df.columns

Index(['Unnamed: 0', 'hosp', 'hosp.1', 'hosp.2', 'rea', 'rea.1', 'rea.2',
       'rad', 'rad.1', 'rad.2', 'dc', 'dc.1', 'dc.2', 'season_autumn',
       'season_spring', 'season_summer', 'season_winter'],
      dtype='object')

In [ ]:

# Créer une liste des colonnes catégorielles
categorical_columns = ['season_autumn',
       'season_spring', 'season_summer', 'season_winter']

# Créer des colonnes hot encoder pour chaque colonne catégorielle
for column in categorical_columns:
    df = pd.get_dummies(df, columns=[column])

# Afficher les 5 premières lignes du DataFrame
df.head()

,Unnamed: 0,hosp,hosp.1,hosp.2,rea,rea.1,rea.2,rad,rad.1,rad.2,...,dc.1,dc.2,season_autumn_False,season_autumn_True,season_spring_False,season_spring_True,season_summer_False,season_summer_True,season_winter_False,season_winter_True
4,0,5226.0,2912.0,2134.0,1297.0,905.0,362.0,1587.0,777.0,764.0,...,250.0,190.0,True,False,True,False,True,False,False,True
5,1,5900.0,3298.0,2401.0,1453.0,1018.0,401.0,1811.0,881.0,888.0,...,297.0,219.0,True,False,False,True,True,False,True,False
6,2,6954.0,3906.0,2815.0,1674.0,1176.0,461.0,2117.0,1031.0,1040.0,...,356.0,263.0,True,False,False,True,True,False,True,False
7,3,8673.0,4882.0,3532.0,2080.0,1465.0,569.0,2567.0,1270.0,1247.0,...,497.0,350.0,True,False,False,True,True,False,True,False
8,4,10163.0,5843.0,4149.0,2503.0,1773.0,694.0,3281.0,1609.0,1624.0,...,661.0,425.0,True,False,False,True,True,False,True,False


In [ ]:
# Afficher les valeurs nulles dans le DataFrame
df.isnull().sum()

Unnamed: 0       0
hosp             0
hosp.1           0
hosp.2           0
rea              0
rea.1            0
rea.2            0
rad              0
rad.1            0
rad.2            0
dc               0
dc.1             0
dc.2             0
season_autumn    0
season_spring    0
season_summer    0
season_winter    0
dtype: int64

In [ ]:


X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=["dc"]), df["dc"], test_size=0.2, random_state=42)


In [ ]:
df.columns

Index(['Unnamed: 0', 'hosp', 'hosp.1', 'hosp.2', 'rea', 'rea.1', 'rea.2',
       'rad', 'rad.1', 'rad.2', 'dc', 'dc.1', 'dc.2', 'season_autumn_False',
       'season_autumn_True', 'season_spring_False', 'season_spring_True',
       'season_summer_False', 'season_summer_True', 'season_winter_False',
       'season_winter_True'],
      dtype='object')

In [ ]:
X_train.shape

(885, 20)

In [ ]:


# Normaliser les données numériques
# scaler = StandardScaler()
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
X_train

array([[0.85081374, 0.51740692, 0.44410616, ..., 0.        , 1.        ,
        0.        ],
       [0.69981917, 0.63331491, 0.57224171, ..., 0.        , 1.        ,
        0.        ],
       [0.79837251, 0.39500587, 0.34184923, ..., 1.        , 1.        ,
        0.        ],
       ...,
       [0.94394213, 0.38464461, 0.3146469 , ..., 0.        , 0.        ,
        1.        ],
       [0.99005425, 0.3055191 , 0.24316632, ..., 0.        , 0.        ,
        1.        ],
       [0.77757685, 0.5517027 , 0.48454564, ..., 1.        , 1.        ,
        0.        ]])